In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import matplotlib

In [9]:
%matplotlib widget

In [10]:
from overfit.trainers.overfit import OverfitTrainer
import matplotlib.pyplot as plt
from PIL import Image
from overfit.utils.img2vid import zigzag, display_video
import torchvision.transforms.functional as FT
from torchvision.models import resnet152, ResNet152_Weights
from torchvision.models import resnet8, ResNet8_Weights
import urllib

In [11]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [13]:
input_image = Image.open("dog.jpg")
input_tensor = FT.to_tensor(input_image)
_, h, w = input_tensor.size()
input_video = zigzag(input_tensor, h // 3, w // 3)
# vid = display_video(input_video)
# vid.save("dog.mp4")
print(len(input_video))

69


In [15]:
def normalize_rgb(img):
    return FT.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_video = [normalize_rgb(frame).unsqueeze(0) for frame in input_video]

In [20]:
srcnet = resnet152(weights=ResNet152_Weights.IMAGENET1K_V1).eval()
tgtnet_trainer = OverfitTrainer()
tgtnet_trainer.set(
    pretrained_classifier=srcnet,
    num_classes=1000,
    confidence=0.1,
    weight_decay=30,
    lr_scale=0.01,
    momentum=0.9,
)
tgtnet_trainer.new_experiment()
tgtnet_trainer.test(input_video, [258] * len(input_video))